In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data

In [4]:
# Hyper-parameters
# SEQ_LENGTH = 10
INPUT_SIZE = 22
HIDDEN_SIZE = 200
NUM_LAYERS = 6
BATCH_SIZE = 100
NUM_EPOCHS = 15
LEARNING_RATE = 0.001

In [5]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.hidden_size = HIDDEN_SIZE
        self.num_layers = NUM_LAYERS
        
        self.lstm = nn.LSTM(
            input_size=INPUT_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            batch_first=True
        )
        self.fc = nn.Linear(HIDDEN_SIZE, 3)
    
    def forward(self, x):
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.fc(out[:, -1, :])
        return out

In [6]:
rnn = RNN()

In [7]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=LEARNING_RATE)

In [8]:
filenames = ['train_data/aalborg.csv', 'train_data/alpine-1.csv', 'train_data/f-speedway.csv']
X_train = None
y_train = None

for file in filenames:
    dataset = pd.read_csv(file)
    
    X = dataset.iloc[:, 3:].values
    y = dataset.iloc[:, :3].values
    
    if X_train is None:
        X_train = X
        y_train = y
    else:
        X_train = np.concatenate((X_train, X))
        y_train = np.concatenate((y_train, y))
        
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
imputer = imputer.fit(X_train)
X_train = imputer.transform(X_train)

In [9]:
X_train = torch.from_numpy(X_train)
X_train = X_train.float()
y_train = torch.from_numpy(y_train)
y_train = y_train.float()

ds = torch.utils.data.TensorDataset(X_train, y_train)

train_loader = torch.utils.data.DataLoader(dataset=ds,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

In [10]:
for epoch in range(NUM_EPOCHS):
    for i, (X, y) in enumerate(train_loader):
        params = Variable(X.view(-1, 1, INPUT_SIZE))
        command = Variable(y)
                
        optimizer.zero_grad()
        outputs = rnn(params)
        loss = criterion(outputs, command)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, NUM_EPOCHS, i+1, len(X_train)//100, loss.data[0]))

Epoch [1/15], Step [100/240], Loss: 0.0832
Epoch [1/15], Step [200/240], Loss: 0.0470
Epoch [2/15], Step [100/240], Loss: 0.0303
Epoch [2/15], Step [200/240], Loss: 0.0240
Epoch [3/15], Step [100/240], Loss: 0.0434
Epoch [3/15], Step [200/240], Loss: 0.0198
Epoch [4/15], Step [100/240], Loss: 0.0466
Epoch [4/15], Step [200/240], Loss: 0.0275
Epoch [5/15], Step [100/240], Loss: 0.0421
Epoch [5/15], Step [200/240], Loss: 0.0117
Epoch [6/15], Step [100/240], Loss: 0.0210
Epoch [6/15], Step [200/240], Loss: 0.0247
Epoch [7/15], Step [100/240], Loss: 0.0331
Epoch [7/15], Step [200/240], Loss: 0.0207
Epoch [8/15], Step [100/240], Loss: 0.0205
Epoch [8/15], Step [200/240], Loss: 0.0668
Epoch [9/15], Step [100/240], Loss: 0.0306
Epoch [9/15], Step [200/240], Loss: 0.0376
Epoch [10/15], Step [100/240], Loss: 0.0352
Epoch [10/15], Step [200/240], Loss: 0.0331
Epoch [11/15], Step [100/240], Loss: 0.0183
Epoch [11/15], Step [200/240], Loss: 0.0294
Epoch [12/15], Step [100/240], Loss: 0.0350
Epoch 

In [11]:
torch.save(rnn.state_dict(), 'rnn_params.pt')

In [12]:
torch.save(rnn, 'whole_net.pt')

/usr/local/miniconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [19]:
ds.data_tensor


  -0.0380   -0.0001    0.0004  ...     5.3198    5.0765    4.9997
  -0.0380   -0.0001    0.0004  ...     5.3198    5.0765    4.9997
  -0.0380   -0.0001    0.0004  ...     5.3198    5.0765    4.9997
             ...                ⋱                ...             
 290.8040   -0.0476   -0.0141  ...    15.2831   14.5443   14.2878
 290.8360   -0.0459   -0.0143  ...    15.3119   14.5711   14.3136
 290.8680   -0.0441   -0.0145  ...     7.5776    7.2365    7.1239
[torch.FloatTensor of size 24024x22]